# Spam Classification
### Used 2 Models Baseline Dense and LSTM

In [1]:
import numpy as np 
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Activation, Embedding, LSTM
import pandas as pd

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_csv("../spam.csv", encoding='iso-8859-1')

In [3]:
print(data.columns.values)

['v1' 'v2' 'Unnamed: 2' 'Unnamed: 3' 'Unnamed: 4']


In [4]:
y_attribute_sub = dict(zip(['spam', 'ham'], [1, 0]))
for i in range(0, len(data)):
    data['v1'][i] = y_attribute_sub[data['v1'][i]]

In [5]:
print(data.head())

  v1                                                 v2 Unnamed: 2 Unnamed: 3  \
0  0  Go until jurong point, crazy.. Available only ...        NaN        NaN   
1  0                      Ok lar... Joking wif u oni...        NaN        NaN   
2  1  Free entry in 2 a wkly comp to win FA Cup fina...        NaN        NaN   
3  0  U dun say so early hor... U c already then say...        NaN        NaN   
4  0  Nah I don't think he goes to usf, he lives aro...        NaN        NaN   

  Unnamed: 4  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  


In [6]:
data = data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1)
print(data.head())

  v1                                                 v2
0  0  Go until jurong point, crazy.. Available only ...
1  0                      Ok lar... Joking wif u oni...
2  1  Free entry in 2 a wkly comp to win FA Cup fina...
3  0  U dun say so early hor... U c already then say...
4  0  Nah I don't think he goes to usf, he lives aro...


Naive Method:

Using Dictionary

In [ ]:
"""corpus = list()
for i in range(0, len(data)):
    corpus.extend(re.sub(r'[^\w]', ' ', data['v2'][i]).strip().split())
corpus = list(set(corpus))
print(len(corpus))"""

In [ ]:
"""encoding = dict(zip(corpus, [i+1 for i in range(0, len(corpus))]))"""

In [ ]:
"""max_length = 0
for i in range(0, len(data)):
    vector = re.sub(r'[^\w]', ' ', data['v2'][i]).strip().split()
    for j in range(0, len(vector)):
        vector[j] = encoding[vector[j]]
    data['v2'][i] = vector
    if(max_length < len(vector)):
        max_length = len(vector)
print(data.head(), max_length)"""

In [ ]:
"""print(data.head())
X = np.zeros((len(data['v2']), max_length))
for i in range(1, len(data)):
    data['v2'][i].extend([0 for j in range(0, max_length-len(data['v2'][i]))])
    X[i,:] = data['v2'][i]
print(data.head())"""

Using Tokenizer from keras with tf-idf

In [7]:
X = data.v2
y = data.v1
le_obj = LabelEncoder()
y = le_obj.fit_transform(y)
y = y.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [8]:
max_length = 150
max_words = 1000
token = Tokenizer(num_words = max_words)
token.fit_on_texts(X_train)
sequences = token.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(sequences, maxlen = max_length)
sequences = token.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(sequences, maxlen = max_length)

In [9]:
def baseline_dense():
    model = Sequential()
    model.add(Dense(100, input_shape=(max_length, ), activation ='relu', name='dense_7'))
    model.add(Dense(30, activation='relu', name='dense_2'))
    model.add(Dense(1, activation='sigmoid', name='dense_3'))
    return model

In [15]:
def lstm():
    model = Sequential()
    model.add(Embedding(max_words, 16, input_length = max_length, name = 'embedding'))
    model.add(LSTM(64, return_sequences = True, name = 'lstm_1'))
    model.add(LSTM(32, return_sequences = False, name = 'lstm_2'))
    model.add(Dense(1, activation='sigmoid', name = 'dense_1'))
    return model

In [16]:
model = lstm()
model.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 16)           16000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 150, 64)           20736     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 49,185
Trainable params: 49,185
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
print("Shape of input: {} and {} and shape of output: {}".format(X_train.shape, len(X_train[0]), y_train[0]))
model.fit(X_train, y_train, epochs=10, batch_size = 128)

Shape of input: (4457, 150) and 150 and shape of output: [0]
Epoch 1/10
4457/4457 [==============================] - 16s 4ms/step - loss: 0.4544 - acc: 0.8600
Epoch 2/10
4457/4457 [==============================] - 15s 3ms/step - loss: 0.1760 - acc: 0.9363
Epoch 3/10
4457/4457 [==============================] - 14s 3ms/step - loss: 0.0756 - acc: 0.9809
Epoch 4/10
4457/4457 [==============================] - 14s 3ms/step - loss: 0.0486 - acc: 0.9881
Epoch 5/10
4457/4457 [==============================] - 14s 3ms/step - loss: 0.0361 - acc: 0.9917
Epoch 6/10
4457/4457 [==============================] - 13s 3ms/step - loss: 0.0274 - acc: 0.9933
Epoch 7/10
4457/4457 [==============================] - 14s 3ms/step - loss: 0.0203 - acc: 0.9960
Epoch 8/10
4457/4457 [==============================] - 13s 3ms/step - loss: 0.0161 - acc: 0.9971
Epoch 9/10
4457/4457 [==============================] - 14s 3ms/step - loss: 0.0138 - acc: 0.9969
Epoch 10/10
4457/4457 [==============================] - 

In [19]:
model.evaluate(X_test, y_test)

1115/1115 [==============================] - 3s 3ms/step


[0.11307094661527284, 0.9766816143497757]

Model with Dense Network

In [10]:
model = baseline_dense()
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 100)               15100     
_________________________________________________________________
dense_2 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 18,161
Trainable params: 18,161
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [11]:
model.fit(X_train, y_train, epochs = 10, batch_size=128)

Epoch 1/10
4457/4457 [==============================] - 1s 138us/step - loss: 2.9594 - acc: 0.8010
Epoch 2/10
4457/4457 [==============================] - 0s 65us/step - loss: 2.1618 - acc: 0.8661
Epoch 3/10
4457/4457 [==============================] - 0s 59us/step - loss: 2.1617 - acc: 0.8661
Epoch 4/10
4457/4457 [==============================] - 0s 60us/step - loss: 2.1616 - acc: 0.8661
Epoch 5/10
4457/4457 [==============================] - 0s 69us/step - loss: 2.1615 - acc: 0.8661
Epoch 6/10
4457/4457 [==============================] - 0s 66us/step - loss: 2.1614 - acc: 0.8661
Epoch 7/10
4457/4457 [==============================] - 0s 58us/step - loss: 2.1613 - acc: 0.8661
Epoch 8/10
4457/4457 [==============================] - 0s 59us/step - loss: 2.1612 - acc: 0.8661
Epoch 9/10
4457/4457 [==============================] - 0s 67us/step - loss: 2.1611 - acc: 0.8661
Epoch 10/10
4457/4457 [==============================] - 0s 61us/step - loss: 2.1610 - acc: 0.8661


In [12]:
model.evaluate(X_test, y_test)

1115/1115 [==============================] - 0s 100us/step


[2.146443904354968, 0.8654708518041089]